# Multi-Head Attention

Multi-Head Attention is an important part of all Transformer-based models.
This tutorial will show how to write it and how to then optimize it.

In [1]:
import time
import torch
import ttnn

torch.manual_seed(0)

device_id = 0
device = ttnn.open(device_id)

                  Metal | INFO     | Initializing device 0
                 Device | INFO     | Opening user mode device driver
2024-01-29 16:47:01.720 | INFO     | SiliconDriver   - Detected 1 PCI device : {0}
2024-01-29 16:47:01.731 | WARNING  | SiliconDriver   - init_detect_tt_device_numanodes(): Could not determine NumaNodeSet for TT device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2024-01-29 16:47:01.731 | WARNING  | SiliconDriver   - Could not find NumaNodeSet for TT Device (physical_device_id: 0 pci_bus_id: 0000:00:08.0)
2024-01-29 16:47:01.732 | WARNING  | SiliconDriver   - bind_area_memory_nodeset(): Unable to determine TT Device to NumaNode mapping for physical_device_id: 0. Skipping membind.
---- ttSiliconDevice::init_hugepage: bind_area_to_memory_nodeset() failed (physical_device_id: 0 ch: 0). Hugepage allocation is not on NumaNode matching TT Device. Side-Effect is decreased Device->Host perf (Issue #893).
                  Metal | INFO     | AI CLK for device 0 is:

## Enable program cache

In [2]:
ttnn.enable_program_cache()

                     Op | INFO     | Program Cache: enabled.


## Write Multi-Head Attention using ttnn

Multi-head can be implemented in `torch` using just 6 operations:
1. `torch.matmul`
2. `torch.add`
3. `torch.reshape`
4. `torch.permute`
5. `torch.mul`
6. `torch.softmax`

`ttnn` provides the exact same APIs to do that and therefore multi-head attention can be implemented as shown below:

In [3]:
def multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    *,
    num_heads,
):
    batch_size, sequence_size, hidden_size = hidden_states.shape
    head_size = hidden_size // num_heads

    query = hidden_states @ query_weight
    query = query + query_bias
    query = ttnn.reshape(query, (batch_size, sequence_size, num_heads, head_size))
    query = ttnn.permute(query, (0, 2, 1, 3))

    key = hidden_states @ key_weight
    key = key + key_bias
    key = ttnn.reshape(key, (batch_size, sequence_size, num_heads, head_size))
    key = ttnn.permute(key, (0, 2, 3, 1))

    value = hidden_states @ value_weight
    value = value + value_bias
    value = ttnn.reshape(value, (batch_size, sequence_size, num_heads, head_size))
    value = ttnn.permute(value, (0, 2, 1, 3))

    attention_scores = query @ key
    attention_scores = attention_scores * (1 / (head_size**0.5))
    attention_probs = ttnn.softmax(attention_scores, dim=-1)

    context_layer = attention_probs @ value
    context_layer = ttnn.permute(context_layer, (0, 2, 1, 3))
    context_layer = ttnn.to_layout(context_layer, layout=ttnn.ROW_MAJOR_LAYOUT)
    context_layer = ttnn.reshape(context_layer, (batch_size, sequence_size, hidden_size))
    context_layer = ttnn.to_layout(context_layer, layout=ttnn.TILE_LAYOUT)

    self_output = context_layer @ output_weight
    self_output = self_output + output_bias

    return self_output

Now that the model is written, let's create input tensors to run it and test it

## Configuration

In [4]:
batch_size = 8
sequence_size = 384
num_heads = 16
head_size = 64
hidden_size = num_heads * head_size

## Initialize activations and weights using torch

In [5]:
torch_hidden_states = torch.randn((batch_size, sequence_size, hidden_size), dtype=torch.bfloat16)
torch_query_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_query_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_key_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_key_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_value_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_value_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)
torch_output_weight = torch.randn((hidden_size, hidden_size), dtype=torch.bfloat16)
torch_output_bias = torch.randn((hidden_size,), dtype=torch.bfloat16)

## Convert activations and weights to ttnn

In [6]:
hidden_states = ttnn.from_torch(torch_hidden_states, layout=ttnn.TILE_LAYOUT, device=device)
query_weight = ttnn.from_torch(torch_query_weight, layout=ttnn.TILE_LAYOUT, device=device)
query_bias = ttnn.from_torch(torch_query_bias, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
key_weight = ttnn.from_torch(torch_key_weight, layout=ttnn.TILE_LAYOUT, device=device)
key_bias = ttnn.from_torch(torch_key_bias, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
value_weight = ttnn.from_torch(torch_value_weight, layout=ttnn.TILE_LAYOUT, device=device)
value_bias = ttnn.from_torch(torch_value_bias, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)
output_weight = ttnn.from_torch(torch_output_weight, layout=ttnn.TILE_LAYOUT, device=device)
output_bias = ttnn.from_torch(torch_output_bias, layout=ttnn.TILE_LAYOUT, device=device, memory_config=ttnn.L1_MEMORY_CONFIG)

                     Op | INFO     | Finished Operation ttnn.from_torch                                    in          157919 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in          615497 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           77089 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_device                                     in          188479 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           39650 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           29399 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_layout                                     in          609147 nanoseconds
                     Op | INFO    

## Run the first iteration of Multi-Head Attention

In [7]:
start = time.time()
multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    num_heads=num_heads,
)
end = time.time()
duration = end - start

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           28750 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           20290 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Matmul                               in       512423586 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Matmul                               in       512662125 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           98239 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           24540 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           21170 nanoseconds
                     Op | INFO    

In [8]:
print(f"Multi-head attention ran in {duration} seconds for the first iteration")

Multi-head attention ran in 8.738018274307251 seconds for the first iteration


## Run a subsequent iteration of Multi-Head Attention

In [9]:
start = time.time()
output = multi_head_attention(
    hidden_states,
    query_weight,
    query_bias,
    key_weight,
    key_bias,
    value_weight,
    value_bias,
    output_weight,
    output_bias,
    num_heads=num_heads,
)
end = time.time()
duration = end - start

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           33830 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           22570 nanoseconds
                     Op | INFO     | Finished Program   tt::tt_metal::Matmul                               in         1142623 nanoseconds
                     Op | INFO     | Finished Operation tt::tt_metal::Matmul                               in         1176083 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           25920 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           20850 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           20760 nanoseconds
                     Op | INFO    

In [10]:
print(f"Multi-head attention ran in {duration} seconds for the subsequent iteration because of the program cache")

Multi-head attention ran in 0.8747236728668213 seconds for the subsequent iteration because of the program cache


## Write optimized version of Multi-Head Attention

Optimized version of the multi-head attention can be written by:
- Tilizing all of the tensors ahead of time
- Using more performant matmuls that fuse bias and specify the number of cores they execute on
- Putting every tensor into L1
- Using bfloat8_b data_type
- Using custom `nlp` operations instead of `ttnn.permute` and `ttnn.reshape`

`ttnn.deallocate` calls are needed because otherwise, the cores on the device will run out of the L1 memory

In [11]:
def optimized_multi_head_attention(
    hidden_states,
    fused_qkv_weight,
    fused_qkv_bias,
    self_output_weight,
    self_output_bias,
    *,
    num_heads,
    num_cores_x=12,
):
    batch_size, _, hidden_size = hidden_states.shape
    head_size = hidden_size // num_heads
    
    hidden_states = ttnn.to_layout(hidden_states, ttnn.TILE_LAYOUT)

    fused_qkv_output = ttnn.linear(
        hidden_states,
        fused_qkv_weight,
        bias=fused_qkv_bias,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat8_b,
        core_grid=(batch_size, num_cores_x),
    )

    (
        query,
        key,
        value,
    ) = ttnn.transformer.split_query_key_value_and_split_heads(
        fused_qkv_output,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        num_heads=num_heads,
    )
    ttnn.deallocate(fused_qkv_output)

    attention_scores = ttnn.matmul(
        query,
        key,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat16,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.deallocate(query)
    ttnn.deallocate(key)

    attention_probs = ttnn.transformer.attention_softmax(attention_scores, attention_mask=None, head_size=head_size)

    context_layer = ttnn.matmul(
        attention_probs,
        value,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat8_b,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.deallocate(attention_probs)

    context_layer = ttnn.transformer.concatenate_heads(
        context_layer,
        memory_config=ttnn.L1_MEMORY_CONFIG,
    )

    self_output = ttnn.linear(
        context_layer,
        self_output_weight,
        bias=self_output_bias,
        memory_config=ttnn.L1_MEMORY_CONFIG,
        dtype=ttnn.bfloat16,
        core_grid=(batch_size, num_cores_x),
    )
    ttnn.deallocate(context_layer)

    return self_output

## Pre-process the parameters of the optimized model

1. Fuse QKV weights and biases
2. Reshape and tilize for the optimized operations using preprocess_linear_weight and preprocess_linear_bias
3. Move to device

In [12]:
from ttnn.model_preprocessing import (
    preprocess_linear_bias,
    preprocess_linear_weight,
)

torch_qkv_weight = torch.cat([torch_query_weight, torch_key_weight, torch_value_weight], dim=-1)
torch_qkv_bias = torch.cat([torch_query_bias, torch_key_bias, torch_value_bias], dim=-1)

qkv_weight = preprocess_linear_weight(torch_qkv_weight.T, dtype=ttnn.bfloat16)
qkv_bias = preprocess_linear_bias(torch_qkv_bias, dtype=ttnn.bfloat16)
output_weight = preprocess_linear_weight(torch_output_weight.T, dtype=ttnn.bfloat16)
output_bias = preprocess_linear_bias(torch_output_bias, dtype=ttnn.bfloat16)

qkv_weight = ttnn.to_device(qkv_weight, device)
qkv_bias = ttnn.to_device(qkv_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)
output_weight = ttnn.to_device(output_weight, device)
output_bias = ttnn.to_device(output_bias, device, memory_config=ttnn.L1_MEMORY_CONFIG)

                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           66409 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           59400 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           28190 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_layout                                     in         1849759 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           22980 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           36660 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_torch                                    in           43029 nanoseconds
                     Op | INFO    

## Run the first iteration of the optimized Multi-Head Attention

In [13]:
start = time.time()
hidden_states = ttnn.to_layout(hidden_states, ttnn.TILE_LAYOUT)
optimized_output = optimized_multi_head_attention(
    hidden_states,
    qkv_weight,
    qkv_bias,
    output_weight,
    output_bias,
    num_heads=num_heads,
)
end = time.time()
duration = end - start

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           28830 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           20500 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           18789 nanoseconds
                     Op | INFO     | Finished Program   tt::operations::primary::Matmul                    in       672655470 nanoseconds
                     Op | INFO     | Finished Operation tt::operations::primary::Matmul                    in       673084318 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in          101549 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           23630 nanoseconds
                     Op | INFO    

In [14]:
print(f"Optimized multi-head attention ran in {duration} seconds for the first iteration")

Optimized multi-head attention ran in 3.960914373397827 seconds for the first iteration


## Run a subsequent iteration of the optimized Multi-Head Attention

In [15]:
start = time.time()
optimized_output = optimized_multi_head_attention(
    hidden_states,
    qkv_weight,
    qkv_bias,
    output_weight,
    output_bias,
    num_heads=num_heads,
)
end = time.time()
duration = end - start

                     Op | INFO     | Finished Operation ttnn.reshape                                       in           33830 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           25629 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           21580 nanoseconds
                     Op | INFO     | Finished Program   tt::operations::primary::Matmul                    in          375878 nanoseconds
                     Op | INFO     | Finished Operation tt::operations::primary::Matmul                    in          443458 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           26080 nanoseconds
                     Op | INFO     | Finished Operation ttnn.reshape                                       in           20670 nanoseconds
                     Op | INFO    

In [16]:
print(f"Optimized multi-head attention ran in {duration} seconds for the subsequent iteration because of the program cache")

Optimized multi-head attention ran in 0.007959365844726562 seconds for the subsequent iteration because of the program cache


Note that the optimized multi-head attention is 2 orders of magnitude faster than the initial version

## Check that the output of the optimized version matches the output of the original implementation

In [17]:
torch_output = ttnn.to_torch(output)
torch_optimized_output = ttnn.to_torch(optimized_output)

assert torch.allclose(torch_output, torch_optimized_output)

                     Op | INFO     | Finished Operation ttnn.from_device                                   in         3573149 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_torch                                      in          210268 nanoseconds
                     Op | INFO     | Finished Operation ttnn.from_device                                   in         3408430 nanoseconds
                     Op | INFO     | Finished Operation ttnn.to_torch                                      in          166269 nanoseconds


## Close the device

In [18]:
ttnn.close(device)

                  Metal | INFO     | Closing device 0
